# 作業

實作各種不一樣的方式來調整飽和 / 對比 / 明亮

1. 改變 color space 來調整飽和度
2. 實作直方圖均衡
3. alpha/ beta 調整對比 / 明亮

In [2]:
import cv2
import numpy as np

img_path = 'lena.png'

# 以彩色圖片的方式載入
img = cv2.imread(img_path, cv2.IMREAD_COLOR)

## 改變飽和度

- 轉換成 HSV color space, 改變 s channel 的值

In [29]:
# 轉換BGR 至 HSV、HSL(HLS)
img_hls = cv2.cvtColor(img, cv2.COLOR_BGR2HLS)
img_hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)

# 0-255 → 0-1
img_hsv = img_hsv.astype('float32')
img_hls = img_hls.astype('float32')

In [30]:
# 改變飽和度
img_hsv[:,:,1] = img_hsv[:,:,1]/255 - .2
img_hls[...,-1] = img_hls[...,-1]/255 +.2
img_hsv[img_hsv[:,:,1] < 0] = 0
img_hls[img_hls[...,-1] < 0] = 0

# 轉回0-255、BGR
img_hsv[...,1] = img_hsv[...,1]*255
img_hls[...,-1] = img_hls[...,-1]*255
img_hsv = img_hsv.astype('uint8')
img_hls = img_hls.astype('uint8')
img_bgr1 = cv2.cvtColor(img_hsv, cv2.COLOR_HSV2BGR)
img_bgr2 = cv2.cvtColor(img_hls, cv2.COLOR_HLS2BGR)

#Show image
img_change = np.hstack((img, img_bgr1, img_bgr2))
cv2.imshow('img_bgr', img_change)
cv2.waitKey()
cv2.destroyAllWindows()

## 直方圖均衡

1. case 1: 把彩圖拆開對每個 channel 個別做直方圖均衡再組合起來
2. case 2: 轉換 color space 到 HSV 之後對其中一個 channel 做直方圖均衡

In [31]:
b = cv2.equalizeHist(img[:,:,0])
g = cv2.equalizeHist(img[:,:,1])
r = cv2.equalizeHist(img[:,:,2])

In [32]:
# case 1
# 每個 channel 個別做直方圖均衡
equalHist_by_channel = img.copy()
equalHist_by_channel[:,:,0] = b
equalHist_by_channel[:,:,1] = g
equalHist_by_channel[:,:,2] = r

In [39]:
# case 2 - 轉換 color space 後只對其中一個 channel 做直方圖均衡
img_hsv_equal = img.copy()
img_hsv_equal = cv2.cvtColor(img_hsv_equal, cv2.COLOR_BGR2HSV)
img_hsv_equal[:,:,2] = cv2.equalizeHist(img_hsv_equal[:,:,2])
img_hsv_equal = cv2.cvtColor(img_hsv_equal, cv2.COLOR_HSV2BGR)

# 組合圖片 + 顯示圖片
img_bgr_equalHist = np.hstack((img, equalHist_by_channel, img_hsv_equal))
while True:
    # 比較 (原圖, BGR color space 對每個 channel 做直方圖均衡, HSV color space 對明度做直方圖均衡)
    cv2.imshow('bgr equal histogram', img_bgr_equalHist)
    k = cv2.waitKey(0)
    if k == 27:
        cv2.destroyAllWindows()
        break

## 調整對比 / 明亮

In [42]:
# alpha: 控制對比度 (1.0~3.0)
# beta: 控制明亮度 (0~255)
add_contrast = cv2.convertScaleAbs(img, alpha = 2., beta=0)
add_lighness = cv2.convertScaleAbs(img, alpha = 1., beta=50)

# another method
add_contrast1 = img.copy()
add_lighness1 = img.copy()
for b in range(img.shape[0]):
    for g in range(img.shape[1]):
        for r in range(img.shape[2]):
            add_contrast1[b,g,r] = np.clip(2.*add_contrast1[b,g,r]+0., 0, 255)
            add_lighness[b,g,r] = np.clip(1.*add_contrast1[b,g,r]+50., 0, 255)

# 組合圖片 + 顯示圖片
img_contrast_light = np.hstack((img, add_contrast1, add_lighness1))
while True:
    # 比較不同程度的對比 / 明亮
    cv2.imshow('adjust contrast and brighness', img_contrast_light)
    k = cv2.waitKey(0)
    if k == 27:
        cv2.destroyAllWindows()
        break